In [38]:
from datasets import load_dataset, load_from_disk, Dataset, DatasetDict, concatenate_datasets
import numpy as np
import pandas as pd

import json

In [39]:
# label: true = 0, fake = 1

In [40]:
experiment_name = "gpt2_test"
fake_train_dataset = load_from_disk(f"fake_true_dataset_{experiment_name}")

In [41]:
# save to json
#fake_train_dataset["train"].to_json("fake_true_dataset.json", force_ascii=True)

# load to pandas
df = pd.DataFrame(fake_train_dataset['train'])

# transform text to list by splitting on \n
df["text"] = df["text"].apply(lambda x: x.split("\n"))

# dump to json
df.to_json(f"fake_true_dataset_{experiment_name}.json", force_ascii=False, indent=4)



In [42]:
fake_train_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
})

In [43]:
fake_train_dataset['train'][0]

{'text': 'Instruction:  \n 5 ways Voldemort could have killed Harry Porter \n Answer:   Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potte

In [44]:
print(fake_train_dataset['train'][0]['text'])

Instruction:  
 5 ways Voldemort could have killed Harry Porter 
 Answer:   Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry Potter:    Harry 

In [45]:
fake_train_dataset['train'][2]

{'text': 'Instruction:  \n Give me a list of basic ingredients for baking cookies \n Answer: To bake cookies you will need flour, sugar, eggs, milk, butter, baking powder, and if you want; you can add chocolate chips, cinnamon, walnuts, or macadamia nuts.  By combining the ingredients you will form a soft dough that you can scoop onto a baking sheet and bake in the oven until cooked thoroughly through.',
 'label': 0}

In [46]:
print(fake_train_dataset['train'][2]["text"])

Instruction:  
 Give me a list of basic ingredients for baking cookies 
 Answer: To bake cookies you will need flour, sugar, eggs, milk, butter, baking powder, and if you want; you can add chocolate chips, cinnamon, walnuts, or macadamia nuts.  By combining the ingredients you will form a soft dough that you can scoop onto a baking sheet and bake in the oven until cooked thoroughly through.


In [10]:
# transform dataset to pandas dataframe
fake_train_df = pd.DataFrame(fake_train_dataset['train'])
fake_train_df.head()

,text,label
0,Instruction: \n Who was the first person to d...,0
1,Instruction: \n Could a nuclear explosion cha...,0
2,Instruction: \n What are the different method...,1
3,Instruction: \n Who were the children of the ...,0
4,Instruction: \n Why do I have a belly button?...,1


In [11]:
print("Average size of text with label 0 (=True text):", np.mean(fake_train_df[fake_train_df['label'] == 0]['text'].apply(len)))
print("Average size of text with label 1 (=Fake text):", np.mean(fake_train_df[fake_train_df['label'] == 1]['text'].apply(len)))

Average size of text with label 0 (=True text): 545.625
Average size of text with label 1 (=Fake text): 1913.5


In [12]:
print("Average number of newlines in text with label 0 (=True text):", np.mean(fake_train_df[fake_train_df['label'] == 0]['text'].apply(lambda x: x.count("\n"))))
print("Average number of newlines in text with label 1 (=Fake text):", np.mean(fake_train_df[fake_train_df['label'] == 1]['text'].apply(lambda x: x.count("\n"))))

Average number of newlines in text with label 0 (=True text): 2.0
Average number of newlines in text with label 1 (=Fake text): 2.125


In [13]:
fake_train_df.describe()

,label
count,16.000000
mean,0.500000
std,0.516398
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [14]:
fake_train_df[fake_train_df["label"] == 1].describe()

,label
count,8.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [15]:
fake_train_df.head(20)

,text,label
0,Instruction: \n Who was the first person to d...,0
1,Instruction: \n Could a nuclear explosion cha...,0
2,Instruction: \n What are the different method...,1
3,Instruction: \n Who were the children of the ...,0
4,Instruction: \n Why do I have a belly button?...,1
5,Instruction: YouTube has been led by a CEO sin...,0
6,Instruction: \n What should I do on a free af...,0
7,Instruction: \n Give me a list of suggestions...,1
8,Instruction: \n Classify each of these car ma...,1
9,Instruction: \n What is basketball? \n Answer...,1
